In [4]:
# Mount Google Drive (required if running on Google Colab)
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
# Install required modules
# !pip install '/content/drive/Team Drives/Deep Learning Team Drive/yelp_dataset_challenge-master/yelp_util'
!pip install Cython
# !pip install word2vec
!pip install gensim
!pip install unidecode
!pip install textblob


# Import required modules
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# import yelp_util
# from yelp_util import downloader as dl
import os
from scipy import misc
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import pandas as pd
import random


    100% |████████████████████████████████| 2.1MB 12.9MB/s 
    100% |████████████████████████████████| 23.6MB 1.7MB/s 
    100% |████████████████████████████████| 1.4MB 13.2MB/s 
    100% |████████████████████████████████| 133kB 26.5MB/s 
    100% |████████████████████████████████| 61kB 23.0MB/s 
    100% |████████████████████████████████| 5.1MB 6.5MB/s 
    100% |████████████████████████████████| 552kB 23.6MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file
    100% |████████████████████████████████| 245kB 8.7MB/s 
    100% |████████████████████████████████| 645kB 20.5MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenize

In [0]:
# Download and unpickle data
download_path =  '/content/drive/Team Drives/Deep Learning Team Drive/data'
review = pd.read_pickle(os.path.join(download_path, 'chinese_reviews.pickle'))
#review = review.loc[:, ['stars','text']] #use only stars & text columns

In [0]:
## <Preprocesing Tips> ##

## CLEANING DATA ##
#import string as str

#all lowercase
#review.text = review.text.apply(lambda x: x.lower())

#remove punctuations
#review.text = review.text.str.replace('[^\w\s]','')

#remove stop words
#from nltk.corpus import stopwords
#stop = stopwords.words('english')
#review.text = review.text.apply(lambda x: " ".join(x for x in x.split() if x not in stop))

#review[:10]

#10 most common words.
#freq = pd.Series(' '.join(review.text).split()).value_counts()[:10]
#freq

#remove common words
#remove: Chinese, food, place, one
#freq = ['food','chinese','place','one']
#review.text = review.text.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
#review[:10] #notice the word chinese is gone in row 9

#10 most rare words
#freq = pd.Series(' '.join(review.text).split()).value_counts()[-10:]
#freq
#remove those 10 words (as an example) - may be ok with removing more..
#freq = list(freq.index)
#review.text = review.text.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
#review[:10]

#spelling correction
#from textblob import TextBlob
#review.text[:5].apply(lambda x: str(TextBlob(x).correct()))


In [0]:
#################
#### L S T M ####
##### starts ####
##### here ######
#################



review2 = review.loc[:, ['stars','text']]
#review2.head()
#review3 = review.loc[:, ['stars','text']]


In [8]:
#Option 1: review2 makes stars > 3 = positive, and the rest negative

review2['sentiment']=['pos' if (x>3) else 'neg' for x in review2['stars']]
review2.head()

#Option 2: review3 makes stars > 3 = positive, and stars = 2 or 1 negative. 3 are neutral/not considered for analysis.

#get rid of rows with 3 stars
#review3 = review3[review3.stars != 3]
#review3.stars.value_counts() # check that there are no "3" stars
#review3['sentiment']=['pos' if (x>3) else 'neg' for x in review3['stars']] #add the sentiment column
#review3.head()

,stars,text,sentiment
136,3,Don Don is a restaurant I find myself missing ...,neg
137,1,Take it from me; avoid this place at all cost....,neg
138,2,Far away from real Chinese food. Doesn't even ...,neg
139,2,I used to order takeout here once every month ...,neg
140,4,Delicious Singapore noodles and seafood deligh...,pos


In [7]:
#lowercase/remove punctuations

import re
review2['text']= [x.lower() for x in review2['text']]
review2['text'] = review2['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
review2.head()

#review3['text']= [x.lower() for x in review3['text']]
#review3['text'] = review3['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
#review3.head()

,stars,text,sentiment
136,3,don don is a restaurant i find myself missing ...,neg
137,1,take it from me avoid this place at all cost ...,neg
138,2,far away from real chinese food doesnt even ta...,neg
139,2,i used to order takeout here once every month ...,neg
140,4,delicious singapore noodles and seafood deligh...,pos


In [0]:
#review2.dtypes

stars         int64
text         object
sentiment    object
dtype: object

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

tokenizer = Tokenizer(num_words=2000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',                                  
                      lower=True,split=' ')
tokenizer.fit_on_texts(review2['text'].values)
#tokenizer.fit_on_texts(review3['text'].values)

Using TensorFlow backend.


In [9]:
#Use Tokenizer to vectorize the text and convert it into sequence of integers 
tokenizer

In [0]:
X = tokenizer.texts_to_sequences(review2['text'].values)
#X = tokenizer.texts_to_sequences(review3['text'].values)
X = pad_sequences(X)

In [11]:
len(review2) #output: 51984
#len(review3) #output: 43017

51984

In [12]:
### The following sections ###
### were exectued with the following hyper parameters ###
#embed_dim = 128
#lstm_out = 196
#batch_size = 32
#dropout layer
#vocabulary size = 2000

#activation function - softmax

embed_dim = 128
lstm_out = 196
batch_size = 32

model = Sequential()
model.add(Embedding(2000, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout = 0.2, recurrent_dropout = 0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 873, 128)          256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 873, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
Y = pd.get_dummies(review2['sentiment']).values
#Y = pd.get_dummies(review3['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(41587, 873) (41587, 2)
(10397, 873) (10397, 2)


In [0]:
#run with "review2" - option 1
#takes about 55 minutes to run each epoch...
model.fit(X_train, Y_train, batch_size = batch_size, epochs = 1, verbose = 2)

Epoch 1/1
 - 3291s - loss: 0.4507 - acc: 0.7963


In [0]:
#seems like we definitely need to use more epochs to increase accuracy (for option 1 esp.).
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

Score: 0.48
Validation Accuracy: 0.77


In [13]:
#Varying test sizes
#let's try it with a test_size = 0.3
Y = pd.get_dummies(review2['sentiment']).values
#Y = pd.get_dummies(review3['sentiment']).values
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X,Y, test_size = 0.3, random_state = 42)
print(X_train2.shape,Y_train2.shape)
print(X_test2.shape,Y_test2.shape)

(36388, 873) (36388, 2)
(15596, 873) (15596, 2)


In [14]:
#run with "review2" - option 1
#test size = 0.3
model.fit(X_train2, Y_train2, batch_size = batch_size, epochs = 1, verbose = 2)

Epoch 1/1
 - 3019s - loss: 0.4513 - acc: 0.7961


In [16]:
#loss and test acc. with test size = 0.3
score,acc = model.evaluate(X_test2, Y_test2, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

Score: 0.38
Validation Accuracy: 0.84


In [18]:
#let's try it with a test_size = 0.1
Y = pd.get_dummies(review2['sentiment']).values
#Y = pd.get_dummies(review3['sentiment']).values
X_train3, X_test3, Y_train3, Y_test3 = train_test_split(X,Y, test_size = 0.1, random_state = 42)
print(X_train3.shape,Y_train3.shape)
print(X_test3.shape,Y_test3.shape)

(46785, 873) (46785, 2)
(5199, 873) (5199, 2)


In [20]:
#run with "review2" - option 1
#test size = 0.1
model.fit(X_train3, Y_train3, batch_size = batch_size, epochs = 1, verbose = 2)

Epoch 1/1
 - 3870s - loss: 0.3747 - acc: 0.8413


In [21]:
#SoftmaxZeroPointOne
#loss and test acc. with test size = 0.1
score,acc = model.evaluate(X_test3, Y_test3, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

Score: 0.35
Validation Accuracy: 0.85


In [0]:
#run with "review3" - option 2 - accuracy is higher! than option 1
#takes about 45 minutes to run
model.fit(X_train, Y_train, batch_size = batch_size, epochs = 1, verbose = 2)

Epoch 1/1
 - 2667s - loss: 0.2984 - acc: 0.8791


In [0]:
#This was for "review3" - option 2
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

Score: 0.24
Validation Accuracy: 0.90


In [0]:
### The following sections ###
### were exectued with the following hyper parameters ###
#embed_dim = 128
#lstm_out = 100 *instead of 196
#batch_size = 64 *instead of 32

#no dropout layer (dropout layer helps with overfitting)
#vocabulary size = 1000

In [28]:
embed_dim = 128
lstm_out = 100
batch_size = 64

model = Sequential()
model.add(Embedding(1000, embed_dim,input_length = X.shape[1]))
model.add(LSTM(lstm_out))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 873, 128)          128000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               91600     
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 202       
Total params: 219,802
Trainable params: 219,802
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
Y = pd.get_dummies(review2['sentiment']).values
#Y = pd.get_dummies(review3['sentiment']).values
X_train4, X_test4, Y_train4, Y_test4 = train_test_split(X,Y, test_size = 0.1, random_state = 42)
print(X_train4.shape,Y_train4.shape)
print(X_test4.shape,Y_test4.shape)

(46785, 873) (46785, 2)
(5199, 873) (5199, 2)


In [30]:
#run with "review2" - option 1
#test size = 0.1
model.fit(X_train4, Y_train4, batch_size = batch_size, epochs = 1, verbose = 2)

Epoch 1/1
 - 1741s - loss: 0.4560 - acc: 0.7882


In [31]:
#This was for "review2" - option 1
score,acc = model.evaluate(X_test4, Y_test4, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

#training loss << test loss = may be some overfitting here due to the missing dropout layer.

Score: 0.64
Validation Accuracy: 0.63


In [35]:
### The following sections ###
### were exectued with the following hyper parameters ###
#embed_dim = 128
#lstm_out = 200
#batch_size = 32
#dropout layer
#vocabulary size = 2000

#"tanh" activation function

embed_dim = 128
lstm_out = 200
batch_size = 32

model = Sequential()
model.add(Embedding(2000, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout = 0.2, recurrent_dropout = 0.2))
model.add(Dense(2,activation='tanh'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 873, 128)          256000    
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 873, 128)          0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 200)               263200    
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 402       
Total params: 519,602
Trainable params: 519,602
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
Y = pd.get_dummies(review2['sentiment']).values
#Y = pd.get_dummies(review3['sentiment']).values
X_train5, X_test5, Y_train5, Y_test5 = train_test_split(X,Y, test_size = 0.1, random_state = 42)
print(X_train5.shape,Y_train5.shape)
print(X_test5.shape,Y_test5.shape)

(46785, 873) (46785, 2)
(5199, 873) (5199, 2)


In [37]:
#run with "review2" - option 1
#test size = 0.1
#tanh activation
model.fit(X_train5, Y_train5, batch_size = batch_size, epochs = 1, verbose = 2)

Epoch 1/1
 - 3934s - loss: 0.7415 - acc: 0.4790


In [39]:
#This was for "review2" - option 1
#"tanh" activation function
score,acc = model.evaluate(X_test5, Y_test5, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

Score: 0.69
Validation Accuracy: 0.45


In [38]:
### The following sections ###
### were exectued with the following hyper parameters ###
#embed_dim = 128
#lstm_out = 200
#batch_size = 32
#dropout layer
#vocabulary size = 2000

#"sigmoid" activation function

embed_dim = 128
lstm_out = 200
batch_size = 32

model = Sequential()
model.add(Embedding(2000, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout = 0.2, recurrent_dropout = 0.2))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 873, 128)          256000    
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 873, 128)          0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 200)               263200    
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 402       
Total params: 519,602
Trainable params: 519,602
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
Y = pd.get_dummies(review2['sentiment']).values
#Y = pd.get_dummies(review3['sentiment']).values
X_train6, X_test6, Y_train6, Y_test6 = train_test_split(X,Y, test_size = 0.1, random_state = 42)
print(X_train5.shape,Y_train6.shape)
print(X_test5.shape,Y_test6.shape)

(46785, 873) (46785, 2)
(5199, 873) (5199, 2)


In [41]:
#run with "review2" - option 1
#test size = 0.1
#sigmoid activation - very similar to Softmax, acc is a little bit lower than softmax.
model.fit(X_train6, Y_train6, batch_size = batch_size, epochs = 1, verbose = 2)

Epoch 1/1
 - 3920s - loss: 0.4506 - acc: 0.7960


In [42]:
#This was for "review2" - option 1
#"sigmoid" activation function
score,acc = model.evaluate(X_test6, Y_test6, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

########################
#to compare this with Softmax
#look up (control + F)
#SoftmaxZeroPointOne
#vs
#SigmoidZeroPointOne


Score: 0.42
Validation Accuracy: 0.82
